Минимизация **суммарного** времени ожидания на одном станке

In [1]:
# Структура для хранения задачи: исходный индекс и время выполнения
'''struct Task {
    int id;
    int duration;
};'''
tasks = [(0, 8), (1, 4), (2, 7), (3, 1), (4, 3), (5, 2)]

n = len(tasks)

# ЖАДНЫЙ АЛГОРИТМ: сортировка по возрастанию длительности
tasks.sort(key=lambda x:x[1])
tasks

[(3, 1), (5, 2), (4, 3), (1, 4), (2, 7), (0, 8)]

In [4]:
#Вывод оптимального расписания
currentTime = 0
totalCompletionTime = 0

for task in tasks:
    currentTime += task[1]              # Время завершения текущей задачи
    totalCompletionTime += currentTime; # Добавляем к суммарному времени
    print(f"Задача {task[0]} время: {task[1]}, завершение в: {currentTime}")
print(f"Суммарное время ожидания завершения: {totalCompletionTime}")



Задача 3 время: 1, завершение в: 1
Задача 5 время: 2, завершение в: 3
Задача 4 время: 3, завершение в: 6
Задача 1 время: 4, завершение в: 10
Задача 2 время: 7, завершение в: 17
Задача 0 время: 8, завершение в: 25
Суммарное время ожидания завершения: 62


Минимизация времени выполнения на нескольких станках

In [19]:
# Структура для станка: текущая нагрузка и список задач
class Machine:
    def __init__(self):
        self.totalTime = 0
        self.taskIds = []
    def __repr__(self):
        return f"Machine(Time: {self.totalTime}, tasks: {self.taskIds})"

# Для очереди с приоритетом: нам нужен станок с минимальной нагрузкой
class ComparableMachine:
    def __init__(self, machine):
        self.machine = machine
    def __repr__(self) -> str:
        return f"{self.machine}"

    def __lt__(self, other):
        # Здесь реализуем логику сравнения
        # Например, сравниваем по приоритету
        return self.machine.totalTime < other.machine.totalTime

tasks = [(0, 5), (1, 7), (2, 5), (3, 9), (4, 2), (5, 3), (6, 4), (7, 1)]
numMachines = 3
numTasks = len(tasks)

#ЖАДНЫЙ АЛГОРИТМ LPT
# Шаг 1: Сортировка задач по убыванию длительности
tasks.sort(key=lambda x: -x[1])
tasks

[(3, 9), (1, 7), (0, 5), (2, 5), (6, 4), (5, 3), (4, 2), (7, 1)]

In [20]:
#Шаг 2: Инициализация станков с помощью min-heap
import heapq
machineQueue = []
for i in range(numMachines):
    heapq.heappush(machineQueue, ComparableMachine(Machine()))
machineQueue

[Machine(Time: 0, tasks: []),
 Machine(Time: 0, tasks: []),
 Machine(Time: 0, tasks: [])]

In [21]:
 # Шаг 3: Распределение задач
machines = [0] * numMachines

for task in tasks:
    # Берем станок с минимальной текущей нагрузкой
    leastLoaded = heapq.heappop(machineQueue).machine
    # Назначаем задачу на этот станок
    leastLoaded.totalTime += task[1]
    leastLoaded.taskIds.append(task[0])

    # Возвращаем станок назад в очередь
    heapq.heappush(machineQueue, ComparableMachine(leastLoaded))
machineQueue

[Machine(Time: 12, tasks: [3, 5]),
 Machine(Time: 12, tasks: [0, 2, 4]),
 Machine(Time: 12, tasks: [1, 6, 7])]

In [22]:
# Собираем итоговые данные из очереди
# Для этого нужно извлечь все элементы (они будут не отсортированы)
result = [item.machine for item in machineQueue]
result

[Machine(Time: 12, tasks: [3, 5]),
 Machine(Time: 12, tasks: [0, 2, 4]),
 Machine(Time: 12, tasks: [1, 6, 7])]

In [23]:
# Находим время работы станков
span = 0
for machine in result:
    if machine.totalTime > span:
        span = machine.totalTime
span

12